In [11]:
import os

from llama_index.llms.ollama import Ollama

MODEL="llama3.2"
PROMPT= "Be a good boy"
DATA_PATH="./kb/kbm1.json"
DATA_DIR = "./kb/"

In [12]:
llm = Ollama(model=MODEL, request_timeout=120)

## single file retrieving

In [13]:
def retrieve_data(query):
    # right now this function doesn't use the query, but it could be used to filter the data later...
    with open(DATA_PATH, 'r') as file:
        file_content = file.read()
    return file_content
    
# Step 3: Define the RAG pipeline with retrieval and generation
def rag_pipeline(query):
    # Retrieve relevant documents
    # retrieved_docs = [retrieve_data(query)]
    data = ""
    data = retrieve_data(query)
    # Build the prompt with the retrieved documents
    prompt = PROMPT + data + query
    # print(prompt)
    # Generate the response with Ollama  
    response = llm.complete(prompt)
    return response  # Extracting the generated answer text

In [ ]:
# Test the RAG pipeline
# query = "Give me insight about the main KPIs?"

query = "Hello"

answer = rag_pipeline(query)
print(answer)

In [ ]:
query = "Which is your favourite machine?"

answer = rag_pipeline(query)
print(answer)

## multi-file delivery

In [ ]:
def retrieve_data(query):
    # Aggregate data from all JSON files in the specified directory
    data_content = ""
    for filename in os.listdir(DATA_DIR):
        if filename.endswith(".json"):
            file_path = os.path.join(DATA_DIR, filename)
            with open(file_path, 'r') as file:
                file_content = file.read()
                data_content += file_content + "\n"  # Append content from each file
    return data_content

# Define the RAG pipeline with retrieval and generation
def rag_pipeline(query):
    # Retrieve relevant documents from all JSON files
    data = retrieve_data(query)
    
    # Build the prompt with the retrieved documents
    prompt = PROMPT + data + query
    # Generate the response with Ollama
    response = llm.complete(prompt)
    return response  # Extracting the generated answer text

In [ ]:
# Test the RAG pipeline
# query = "Give me insight about the main KPIs?"

query = "Hello, i have  a working_time on Large cutting machine 1 of 334, is it good?"

answer = rag_pipeline(query)
print(answer)

# notebooking the script.............


In [166]:
import os
from pathlib import Path
from llama_index.llms.ollama import Ollama
from llama_index.core import VectorStoreIndex
from llama_index.core.embeddings import resolve_embed_model
from llama_index.readers.json import JSONReader
from llama_index.core.node_parser import JSONNodeParser
from llama_index.readers.file import FlatReader

In [167]:
DATA_PATH = "./data/"

# setting up reader, parser, and llm
reader = JSONReader()
# parser = JSONNodeParser()     # if we want to split the documents into nodes
llm = Ollama(model="llama3.2", request_timeout=60.0)

In [ ]:

# creating the documents out of the json files
documents = []
for filename in os.listdir(DATA_PATH):
    if filename.endswith(".json"):
        file_path = os.path.join(DATA_PATH, filename)
        #documents.extend(FlatReader().load_data(Path(file_path)))     # if we want to load the data to then split it into nodes
        documents.extend(reader.load_data(input_file=file_path))

# nodes = parser.get_nodes_from_documents(documents)            # if we want to split documents into nodes


In [196]:
parser = JSONNodeParser()

nodes = parser.get_nodes_from_documents(documents)

In [197]:
documents

[Document(id_='2af681ca-8e25-4aeb-aca8-13001acf3382', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='"machine_id": "Large Capacity Cutting Machine 1",\n"month": "2024-06",\n"KPIs": {\n"average_cycle_time": {\n"average": 0.0,\n"min": 0.0,\n"max": 0.0\n"bad_cycles": {\n"average": 0.0,\n"min": 0.0,\n"max": 0.0\n"consumption": {\n"average": 0.0022754360628985696,\n"min": 0.0,\n"max": 0.28049992033965815\n"consumption_idle": {\n"average": 0.0004284444720048216,\n"min": 0.0,\n"max": 0.002268065000498686\n"consumption_working": {\n"average": 0.003705408855178665,\n"min": 0.0,\n"max": 0.033748348936949925\n"cost": {\n"average": 0.0011976140803471493,\n"min": 0.0,\n"max": 0.1762150443093765\n"cost_idle": {\n"average": 0.0007713563761009156,\n"min": 0.0,\n"max": 0.1762150443093765\n"cost_working": {\n"average": 0.0013039613971528039,\n"min": 0.0,\n"max": 0.013332865828880693\n"cycles": {\n"average": 0.8,\n"min": 0.0,\n"max": 2

In [194]:
print(documents[0].__dict__)

{'id_': '3e28b3a4-94ac-4ea2-aa99-1bdcd6940425', 'embedding': None, 'metadata': {}, 'excluded_embed_metadata_keys': [], 'excluded_llm_metadata_keys': [], 'relationships': {}, 'text': '"machine_id": "Large Capacity Cutting Machine 1",\n"month": "2024-06",\n"KPIs": {\n"average_cycle_time": {\n"average": 0.0,\n"min": 0.0,\n"max": 0.0\n"bad_cycles": {\n"average": 0.0,\n"min": 0.0,\n"max": 0.0\n"consumption": {\n"average": 0.0022754360628985696,\n"min": 0.0,\n"max": 0.28049992033965815\n"consumption_idle": {\n"average": 0.0004284444720048216,\n"min": 0.0,\n"max": 0.002268065000498686\n"consumption_working": {\n"average": 0.003705408855178665,\n"min": 0.0,\n"max": 0.033748348936949925\n"cost": {\n"average": 0.0011976140803471493,\n"min": 0.0,\n"max": 0.1762150443093765\n"cost_idle": {\n"average": 0.0007713563761009156,\n"min": 0.0,\n"max": 0.1762150443093765\n"cost_working": {\n"average": 0.0013039613971528039,\n"min": 0.0,\n"max": 0.013332865828880693\n"cycles": {\n"average": 0.8,\n"min": 0.

In [193]:
nodes

[]

In [135]:
#embed_model = resolve_embed_model("local:BAAI/bge-m3")

In [136]:
#embed_model = resolve_embed_model('local:sentence-transformers/all-MiniLM-L6-v2')

In [187]:
embed_model = resolve_embed_model('local:sentence-transformers/all-mpnet-base-v2')

In [ ]:
# if we work with nodes
#vector_index = VectorStoreIndex.from_documents(nodes, embed_model=embed_model)

In [ ]:
# if we work with documents
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, show_progress=True)

Generating embeddings: 100%|██████████| 4/4 [00:06<00:00,  1.61s/it]


In [165]:
query_engine = vector_index.as_query_engine(llm=llm, similarity_top_k=4)

result = query_engine.query("What is the avg consumption for machine Large Capacity Cutting Machine 1 in the month of 2024-03?")
print(result)

0.0021111835419563543
